In [8]:
%run ItemModel.ipynb
%run APIService.ipynb

In [9]:
import inspect, re
import pandas as pd
import csv

# Class DataCrawler

In [79]:
class DataCrawler(apiServiceDelegate):
    newest = 0
    keyword = "thoi trang nam"
    categoryids = ""
    items_json = []
    all_items = [] # List SHOPPEItem
    itemModel = SHOPPEItem()
    fileName = 'ShoppeItem.tsv'
    noneCategory = True
    APIService = APIService(newest, keyword, categoryids)
    
    i = 1
    def __init__(self, keyword, newest = 0, categoryids = '', max_call = 2):
        if categoryids == '':
            self.noneCategory = True
        else:
            self.noneCategory = False
        self.newest = newest
        self.keyword = keyword
        self.categoryids = categoryids
        self.APIService = APIService(self.newest, self.keyword, self.categoryids)
        self.APIService.delegate = self
        self.max_call=max_call
        pass
    
    def setNewest(self,newest):
        self.newest = newest
        pass
    
    def crawlData(self):
        start = 0
        while True:
            try:
                if start < self.max_call:
                    self.APIService.setNewest(self.newest)
                    check = self.APIService.get()
                    start += 1
                    if check == False: break
                else: 
                    self.parseData()
                    print('Done!')
                    break
            except Exception as e:
                self.parseData()
                print(f'Error: {e}')
                break
        pass
    
    def parseData(self):
        self.all_items.clear()
        for item in self.items_json:
            item = item['item_basic']
            itemModel = SHOPPEItem()
            itemModel.name = item['name']
            itemModel.shop_location = item['shop_location']
            itemModel.ctime = item['ctime'] #time since 1970
            itemModel.time_now = int(time.time())
            itemModel.sell_time = itemModel.time_now - itemModel.ctime
            itemModel.historical_sold = item['historical_sold']
            itemModel.shopee_verified = item['shopee_verified']
            itemModel.discount = item['discount']
            itemModel.is_adult = item['is_adult']
            if len(item['tier_variations']) > 0:
                itemModel.options = 1
                for tier_variation in item['tier_variations']:
                    itemModel.options *= len(tier_variation['options'])
            else:
                itemModel.options = 0
            itemModel.show_official_shop_label_in_title = item['show_official_shop_label_in_title']
            itemModel.rating_star = item['item_rating']['rating_star']
            itemModel.rating_count = item['item_rating']['rating_count']
            itemModel.five_star = itemModel.rating_count[5]
            itemModel.four_star = itemModel.rating_count[4]
            itemModel.three_star = itemModel.rating_count[3]
            itemModel.two_star = itemModel.rating_count[2]
            itemModel.one_star = itemModel.rating_count[1]
            # itemModel.flash_sale = item['flash_sale']
            # itemModel.upcoming_flash_sale = item['upcoming_flash_sale']
            itemModel.price_min = item['price_min']
            itemModel.price_min_before_discount = item['price_min_before_discount']
            
            itemModel.price_max = item['price_max']
            itemModel.price_max_before_discount = item['price_max_before_discount']
            itemModel.price = item['price']
            itemModel.price_before_discount = item['price_before_discount']
            itemModel.coin_earn_label = item['coin_earn_label']
            itemModel.liked_count = item['liked_count']
            itemModel.view_count = item['view_count']
            itemModel.cmt_count = item['cmt_count']
            itemModel.is_preferred_plus_seller = item['is_preferred_plus_seller']
            itemModel.show_free_shipping = item['show_free_shipping']
            itemModel.images = len(item['images'])
            try:
                itemModel.video_info_list = len(item['video_info_list'])
            except Exception as e:
                itemModel.video_info_list = 0
            self.all_items.append(itemModel)
        pass
    
    def saveFile(self, name = fileName):
        self.fileName = name
        file = open(self.fileName, 'w', encoding='utf-8')
 
        file.write(f'category_id\tname\tshop_location\titem_public_time\titem_be_got_time\tsell_time\tshopee_verified\tdiscount\tis_adult\t'
                + f'options\tshow_official_shop_label_in_title\trating_star\tfive_star\tfour_star\tthree_star\ttwo_star\tone_star\t'
                + f'price_min\tprice_min_before_discount\t'
                + f'price_max\tprice_max_before_discount\tprice\tprice_before_discount\t'
                + f'coin_earn_label\tliked_count\tview_count\tcmt_count\t'
                + f'is_preferred_plus_seller\tshow_free_shipping\timages\tvideo_info_list\t'
                + f'historical_sold\n')
    
        # Notice that categoryids is not required. Feel free to get data but uncomment below if you did not pass categoryids argument.
#       
        if self.noneCategory == True:
            self.categoryids = '0'
        
        for item in self.all_items:
                file.write(f'{self.categoryids}\t{item.name}\t{item.shop_location}\t{item.ctime}\t{item.time_now}\t{item.sell_time}\t{item.shopee_verified}\t{item.discount}\t{item.is_adult}\t'
                + f'{item.options}\t{item.show_official_shop_label_in_title}\t{item.rating_star}\t{item.five_star}\t'
                + f'{item.four_star}\t{item.three_star}\t{item.two_star}\t{item.one_star}\t'
                + f'{item.price_min}\t{item.price_min_before_discount}\t'
                + f'{item.price_max}\t{item.price_max_before_discount}\t{item.price}\t{item.price_before_discount}\t'
                + f'{item.coin_earn_label}\t{item.liked_count}\t{item.view_count}\t{item.cmt_count}\t'
                + f'{item.is_preferred_plus_seller}\t{item.show_free_shipping}\t{item.images}\t{item.video_info_list}\t'
                + f'{item.historical_sold}\n')
                
    def updateDataWhenSuccess(self, json_pydata):
        print(len(json_pydata))
        print(f'get! {self.i}')
        
        if json_pydata['items'] != None:
            self.items_json.extend(json_pydata['items'])
        else:
            self.parseData()
            print('get All Data!')
            return False

        self.newest += 50
        self.i += 1
        time.sleep(10)
        return True
        pass
    def getDataError(self, error):
        self.parseData()
        print(f'Error when request: {error}')
        return False
        pass
    
    pass

# Crawling Data

In [80]:
crawler = DataCrawler(newest = 0, keyword = 'thoi trang nam', categoryids = '11035595', max_call=1)

In [81]:
crawler.crawlData()


29
get! 1
Done!


In [17]:
# crawler.parseData()

In [82]:
# Check last request
print(json.dumps(crawler.APIService.json_pydata, indent = 3, sort_keys=True))

{
   "adjust": {
      "count": 3000
   },
   "algorithm": "eyJzZWFyY2giOiIwLmEuNzcxNkBaQVRLSVpIRFhOVldWWkZNS1VFSjBEWVZGSVkxRVNHVkVUMk9QNE9ONjNQUTc1R08xNzUifQ==",
   "autoplay_info": null,
   "batch_size": 50,
   "bff_meta": null,
   "cached_result": null,
   "disclaimer_infos": [],
   "error": null,
   "error_msg": null,
   "experiments": [
      {
         "key": "sch.user_feedback_in_srp_bff",
         "value": "0"
      }
   ],
   "food_item_info": {
      "total_count": 0
   },
   "hint_keywords": null,
   "items": [
      {
         "ads_keyword": "wkdaelpmissisiht",
         "adsid": 33216821,
         "algo_image": null,
         "bff_item_tracking": "{\"label_ids\":[1000279,1007525,44,1400025226,1000742,1002603,1000512,700066157],\"is_on_flash_sale\":false,\"is_liked\":false,\"with_video\":false,\"is_dp_mirror_sku\":false,\"image_position\":null}",
         "biz_json": null,
         "campaign_stock": null,
         "campaignid": 21003301,
         "collection_id": null,
     

In [83]:
# Check feature value
crawler.all_items[0].ctime

1656665738

In [88]:
# Test ParseData
print(crawler.all_items[0].name)
# print(crawler.all_items[50].name)
print(crawler.all_items[1].name)
# print(crawler.all_items[51].name)

Áo Thun Nam Ngắn Tay Cổ Tròn Màu Sắc Thời Trang
ÁO THUN COTTON 4 CHIỀU IN HÌNH I LOVE YOU TỪ BÉ


In [90]:
crawler.saveFile("ShoppeItem_thoiTrangNam_11035595.tsv")

In [91]:
# Test saveFile
items_df = pd.read_csv('ShoppeItem_thoiTrangNam_11035595.tsv', sep='\t')
pd.set_option('max_colwidth', 10000)

items_df



,category_id,name,shop_location,item_public_time,item_be_got_time,sell_time,shopee_verified,discount,is_adult,options,...,price_before_discount,coin_earn_label,liked_count,view_count,cmt_count,is_preferred_plus_seller,show_free_shipping,images,video_info_list,historical_sold
0,11035595,Áo Thun Nam Ngắn Tay Cổ Tròn Màu Sắc Thời Trang,Nước ngoài,1656665738,1665151713,8485975,False,41%,False,18,...,17403000000,None,7,None,10,False,False,8,0,25
1,11035595,ÁO THUN COTTON 4 CHIỀU IN HÌNH I LOVE YOU TỪ BÉ,TP. Hồ Chí Minh,1664905415,1665151713,246298,False,13%,False,5,...,38000000000,None,21,None,0,False,False,5,1,0
2,11035595,Áo thun tay lỡ form rộng nam nữ chữ kiểu SVG vải đẹp dày mịn,Đồng Nai,1619249160,1665151713,45902553,True,None,False,20,...,0,None,2181,None,4593,False,False,3,0,17511
3,11035595,Album 7 - BST áo thun unisex form rộng mùa hè thời trang phong cách Hàn Quốc,Đồng Nai,1555690368,1665151713,109461345,True,None,False,48,...,0,None,44132,None,17193,False,True,8,0,64114
4,11035595,Áo Thun Loui 7 Màu ❤ Áo Phông Cotton Dáng Rộng Nam Nữ NVA71,Hải Dương,1651193170,1665151713,13958543,True,28%,False,9,...,9000000000,None,38,None,12,False,False,5,0,54
5,11035595,"Áo Thun Thể Thao Nam, Áo Tập Gym 215STORE, Vải Thun Mát Nhẹ",TP. Hồ Chí Minh,1656498269,1665151713,8653444,False,36%,False,16,...,17000000000,None,4,None,6,False,False,9,0,11
6,11035595,ÁO THUN TN C.K AT544 - THỜI TRANG NAM VNXK,Khánh Hòa,1664436730,1665151713,714983,False,None,False,12,...,0,None,0,None,0,False,False,6,0,0
7,11035595,"Áo thun phản quang LVT quanh cổ ⚡ Phông tay lỡ dáng thụng rộng nam nữ unisex chất cotton mặc đôi , nhóm cực đẹp",Nam Định,1624808829,1665151713,40342884,True,46%,False,8,...,3900000000,None,2214,None,701,False,False,9,0,2486
8,11035595,BST 50 - Áo thun nam nữ form rộng vải dày mịn logo cá tính cách điệu,Đồng Nai,1589604777,1665151713,75546936,True,None,False,48,...,0,None,7989,None,2307,False,False,8,0,8825
9,11035595,Áo Thun Nam Cổ Tròn Oversize Phong Cách Hàn Quốc,Nước ngoài,1628838671,1665151713,36313042,True,40%,False,18,...,20000000000,None,116,None,81,False,False,4,0,383


In [87]:
# Migrate Data
# Run this if you run the crawler before it've been updated to auto wirte the sell_time feature.

# items_df.insert(4, 'sell_time', items_df['item_be_got_time'] - items_df['item_public_time'])
# items_df.to_csv('ShoppeItem_thoiTrangNam_aoKhoacAoVest_0-949.tsv', sep = '\t', index=False)

In [88]:
del items_df